In [14]:
import re

text = "Hey does this company sell my data is the url: https://github.com/"
pattern = r"https?:\/\/(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}"



def extract_email(text):
    """Extract the url"""

    pattern = r"https?:\/\/(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}"

    match = re.search(pattern, text)
    if match:
        return match.group()

    else:
        return False

d =extract_email(text)


url_check = extract_email(text)

if   not  url_check:
    print("true")


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import html
from backend.src.tools.vector_store import VectorStoreManager

test = VectorStoreManager()
results = test.vectordb_query_chatbot("I don't want to share data with github")

def clean_page_content(text):
    try:
        text = text.encode().decode("unicode_escape")
    except:
        pass
    return html.unescape(text)

print(clean_page_content(results[3].page_content))  # Optional preview


# Evaluation function
def evaluate_retrieval_similarity(self, query: str, results: list):
    print(f"\n🔍 Evaluating similarity for query: '{query}'")
    query_emb = self.embedding_function.embed_query(query)
    doc_embs = [self.embedding_function.embed_query(doc.page_content) for doc in results]

    similarities = cosine_similarity([query_emb], doc_embs)[0]
    scored_docs = list(zip(results, similarities))

    for i, (doc, score) in enumerate(scored_docs):
        print(f"\nResult {i+1}:")
        print(f"Score: {score:.3f}")
        print(f"Preview: {clean_page_content(doc.page_content[:300])}...\n")

    return scored_docs

# Attach the method
from types import MethodType
test.evaluate_retrieval_similarity = MethodType(evaluate_retrieval_similarity, test)

# Evaluate
scored = test.evaluate_retrieval_similarity("I don't want to share data with github", results)


Using local credentials.


/Users/mohammedansari/Desktop/clausebit/.venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


<p>This license does not grant GitHub the right to sell Your Content. It also does not grant GitHub the right to otherwise distribute or use Your Content outside of our provision of the Service, except that as part of the right to archive Your Content, GitHub may permit our partners to store and archive Your Content in public repositories in connection with the <a href="https://archiveprogram.github.com/">GitHub Arctic Code Vault and GitHub Archive Program</a>.</p>
<h3 id="5-license-grant-to-other-users" tabindex="-1"><a class="heading-link" href="#5-license-grant-to-other-users">5. License Grant to Other Users<span class="heading-link-symbol" aria-hidden="true"></span></a></h3>
<p>Any User-Generated Content you post publicly, including issues, comments, and contributions to other Users' repositories, may be viewed by others. By setting your repositories to be viewed publicly, you agree to allow others to view and "fork" your repositories (this means that others may make their own copi

In [7]:
result[4].page_content


'<h4 id="how-do-we-and-our-partners-use-cookies-and-similar-technologies" tabindex="-1"><a class="heading-link" href="#how-do-we-and-our-partners-use-cookies-and-similar-technologies">How do we and our partners use cookies and similar technologies?<span class="heading-link-symbol" aria-hidden="true"></span></a></h4>\n<p>The GitHub Services use cookies and similar technologies for a variety of purposes, including to store your preferences and settings, enable you to sign-in, analyze how our Services perform, track your interaction with the Services, develop inferences, combat fraud, and fulfill other legitimate purposes. Some of these cookies and technologies may be provided by third parties, including service providers and advertising partners. For example, our analytics and advertising partners may use these technologies in our Services to collect personal information (such as the pages you visit, the links you click on, and similar usage information, identifiers, and device informati

In [11]:
from datetime import datetime, timezone

timestamp = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")
timestamp

'2025-07-25T08:15:53.012187Z'

In [1]:
from backend.src.tools.webscraper import chunking

import asyncio
result = asyncio.run(chunking("https://github.com/"))



Chunking the data!
Using local credentials.
extracting from {'text': 'Terms', 'href': 'https://docs.github.com/site-policy/github-terms/github-terms-of-service'}
✅ Playwright succeeded, so this is not the issue
extracting from {'text': 'Privacy', 'href': 'https://docs.github.com/site-policy/privacy-policies/github-privacy-statement'}
✅ Playwright succeeded, so this is not the issue
done
generating the metadata tags with con 


In [3]:
result[2].metadata

{'source_url': 'https://github.com/',
 'domain': 'https://github.com/',
 'scrape_date': '2025-07-25T10:35:55.506460Z',
 'language': 'en',
 'policy_type': 'terms_of_service',
 'section_title': 'Definitions',
 'summary': 'This section defines key terms used throughout the terms of service, such as "Account" and "Organizations".',
 'categories': 'definitions',
 'user_impact_level': 'low'}

In [1]:

from backend.src.tools.webscraper import process_company_policies

url = "https://github.com/"
results = await process_company_policies("https://github.com")


Using local credentials.
 Full page text extracted and saved.
 Full page text extracted and saved.
 Full page text extracted and saved.


In [7]:
import  os
base_dir = os.path.abspath(
            os.path.join(os.path.dirname(__file__), "..", "data_warehouse")
        )
base_dir

NameError: name '__file__' is not defined

In [3]:


def clean_policy_text(text: str) -> str:
    # Remove duplicated navigation and menu content
    lines = text.splitlines()

    # Remove lines that are too short or very generic UI strings
    lines = [line.strip() for line in lines if len(line.strip()) > 20 and not re.match(r'^(Search|Sign up|Open menu|GitHub Docs|Copilot|Home|Site policy|Version)', line)]

    # Collapse back into cleaned text
    cleaned_text = "\n".join(lines)

    # Optional: find first real section
    match = re.search(r'(A\. Definitions|Effective date:)', cleaned_text)
    if match:
        cleaned_text = cleaned_text[match.start():]


    return cleaned_text


In [18]:
 t = [clean_policy_text(page["content"]) for page in results]
 len(t)

3

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)


t =clean_policy_text(results[0]["content"])

# Wrap the content and metadata in lists
chunks = text_splitter.create_documents(
    texts=[t],
    metadatas=None
)


chunks[4].metadata



{}

In [6]:
chunks[0].page_content

'Effective date: November 16, 2020\nWe use these basic terms throughout the agreement, and they have specific meanings. You should know what we mean when we use each of the terms. There\'s not going to be a test on it, but it\'s still useful information.\nAn "Account" represents your legal relationship with GitHub. A “Personal Account” represents an individual User’s authorization to log in to and use the Service and serves as a User’s identity on GitHub. “Organizations” are shared workspaces that may be associated with a single entity or with one or more Users where multiple Users can collaborate across many projects at once. A Personal Account can be a member of any number of Organizations.\nThe “Agreement” refers, collectively, to all the terms, conditions, notices contained or referenced in this document (the “Terms of Service” or the "Terms") and all other operating rules, policies (including the GitHub Privacy Statement, available at\ngithub.com/site/privacy\n) and procedures tha

In [12]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model="gemini-2.0-flash-lite",
    temperature=0.8,
    max_output_tokens= 8000
)


parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", '''You are a policy analyzer that generates structured metadata for pre-chunked legal content (e.g., Terms of Service, Privacy Policies). Your output will be used for semantic retrieval, privacy risk detection, and clause-level reasoning in downstream AI applications.'''),

    ("human", '''Given the following policy text, generate a **JSON object** that includes structured metadata fields.

Use this schema:
- "source_url": The original URL of the policy.
- "domain": Detected company domain from the URL.
- "scrape_date": "2025-07-22T22:08:00Z"
- "language": The detected language (e.g., "en").
- "policy_type": One of: "privacy_policy", "terms_of_service", "cookie_policy", "acceptable_use", "other"
- "risk_tags": List of detected privacy or legal risks (e.g., "data_sharing", "location_tracking", etc.)
- "section_title": Descriptive title for this chunk. If none is found, infer one.
- "summary": A 1–2 sentence plain-English summary of what the chunk means.
- "categories": List of high-level topics (e.g., "data_collection", "user_rights", etc.)
- "user_impact_level": One of "low", "medium", or "high" based on how much the policy affects the user's rights or data.
- "version": Policy version if found (e.g., "v2.3")

Return **only one JSON object**. No markdown, no code formatting, and no extra explanation.

Policy Text:
"""
{data}
"""
''')
])


def enrich_metadata(chunk):

    base_chain = prompt|llm|JsonOutputParser()

    response = base_chain.invoke({"data":chunk.page_content})  # properly passes full message list to

    chunk.metadata = response



# Apply enrichment to all chunks
enriched_chunks = [enrich_metadata(chunk) for chunk in chunks]


In [14]:
chunks[3].metadata

{'source_url': None,
 'domain': None,
 'scrape_date': '2025-07-22T22:08:00Z',
 'language': 'en',
 'policy_type': 'acceptable_use',
 'risk_tags': ['content_removal',
  'user_generated_content',
  'copyright_infringement'],
 'section_title': 'Acceptable Use and User-Generated Content',
 'summary': "Users are responsible for their content and must comply with laws and GitHub's policies. GitHub can remove content and close accounts that violate these terms, and users grant GitHub and other users necessary permissions to display and share the content.",
 'categories': ['acceptable_use',
  'user_generated_content',
  'content_ownership',
  'licensing'],
 'user_impact_level': 'medium',
 'version': None}

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model="gemini-2.0-flash-lite",
    temperature=0.8,
    max_output_tokens= 8000
)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
You are a legal document enhancer. Your job is to take a raw policy document (with partial metadata and full content) and return one or more enhanced chunks, each with completed metadata fields.

Each chunk must:
- A section of the policy as structured in input data, section a makes sense  its own
- Be semantically meaningful (e.g., a section or group of related paragraphs)
- Fill in all missing or placeholder metadata fields using the content

⚠️ Output must be a valid JSON **array of objects** — no markdown, no commentary.

Use the following structure:
[
  {{
    "content": "...",
    "metadata": {{
      "text": "...",
      "source_url": "...",
      "domain": "detect company name ",
      "scrape_date": "...",
      "language": "The detected language (e.g., 'en')",
      "policy_type": "One of: 'privacy_policy', 'terms_of_service', 'cookie_policy', 'acceptable_use', 'other'",
      "risk_tags": ["..."],  // e.g., 'data_sharing', 'location_tracking'
      "section_title": "Descriptive title or heading for this chunk (or infer one)",
      "summary": "1–2 sentence plain-English summary of this chunk",
      "categories": ["..."],  // e.g., 'data_collection', 'user_rights'
      "user_impact_level": "low", "medium", or "high",
      "clauses": [{{"title": "...", "clause": "..."}}],  // Key legal clauses if applicable
      "version": "Policy version if available (e.g., 'v2.3')"
    }}
  }},
  ...
]
        """.strip()
    ),
    (
        "human",
        """
Here is a policy document (including its metadata and full content). Return 1 or more enhanced JSON chunks with completed metadata fields:

{{document}}
        """.strip()
    )
])

base_chain = prompt|llm|StrOutputParser()



output = base_chain.invoke({"document": results[0]})
output


'```json\n[\n  {\n    "content": "At The Honest Company, we are committed to protecting your privacy. This Privacy Policy explains how we collect, use, and disclose information about you when you use our websites, mobile applications, and other online products and services (collectively, the “Services”) or when you otherwise interact with us. \\n\\nWe may change this Privacy Policy from time to time. If we make changes, we will notify you by revising the date at the top of this policy and, in some cases, we may provide you with additional notice (such as adding a statement to our website or sending you a notification). We encourage you to review the Privacy Policy whenever you access the Services or otherwise interact with us to stay informed about our information practices and the ways you can help protect your privacy.\\n\\nBy accessing or using the Services, you agree to the terms and conditions of this Privacy Policy.",\n    "metadata": {\n      "text": "At The Honest Company, we a

In [21]:
import re
import json

# Step 1: Remove any accidental ```json blocks
clean_output = re.sub(r"^```json|```$", "", str(output).strip(), flags=re.MULTILINE)

# Step 2: Parse safely with json.loads
try:
    parsed_output = json.loads(clean_output)
except json.JSONDecodeError as e:
    print("❌ JSON parsing failed:", e)
    print("Raw output was:\n", clean_output[:1000])  # preview
    parsed_output = []


In [22]:
parsed_output[3]["co"]

{'text': 'Information We Collect From Other Sources\n\nWe may also obtain information from other sources. For example, if you create or log into your account through a social media service, we may have access to certain information from that service, such as your name, profile picture, and friends list, in accordance with the authorization procedures determined by such social media service. We may also receive information about you from other sources, including our business partners and third parties.',
 'source_url': None,
 'domain': 'The Honest Company',
 'scrape_date': None,
 'language': 'en',
 'policy_type': 'privacy_policy',
 'risk_tags': ['data_sharing'],
 'section_title': 'Information Collection: From Other Sources',
 'summary': 'The Honest Company may gather information from other sources, including social media services (with user authorization) and third-party partners.',
 'categories': ['data_collection', 'third parties'],
 'user_impact_level': 'medium',
 'clauses': [],
 've

In [25]:
import re
import json
from datetime import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a legal policy enhancer. Each input is a section from a Terms of Service or Privacy Policy document, with partial metadata.

You must fill in the missing metadata, using the content to infer the following:
- language
- policy_type
- risk_tags
- section_title
- summary
- categories
- user_impact_level
- clauses
- version

Return a valid JSON array like:
[
  {{
    "content": "...",
    "metadata": {{
      "text": "...",
      "source_url": "...",
      "domain": "...",
      "scrape_date": "...",
      "language": "...",
      "policy_type": "...",
      "risk_tags": ["..."],
      "section_title": "...",
      "summary": "...",
      "categories": ["..."],
      "user_impact_level": "...",
      "clauses": [{{"title": "...", "clause": "..."}}],
      "version": "..."
    }}
  }}
]
No markdown, only raw JSON.
    """),
    ("human", """
Here is one section of the document:

{{document}}
    """)
])

base_chain = prompt | llm | StrOutputParser()

# -------------------------------
# Step 1: Chunk the full document
# -------------------------------
def split_policy_sections(full_text: str) -> list[str]:
    return re.split(r"\n(?=[A-Z]\. )", full_text)

# -------------------------------
# Step 2: Wrap into LLM-ready input
# -------------------------------
def build_input_chunk(text, section_label, metadata_stub):
    return {
        "content": text.strip(),
        "metadata": {
            **metadata_stub,
            "section_title": section_label,
            "summary": "Fill this in.",
            "risk_tags": [],
            "categories": [],
            "language": "en",
            "policy_type": "unknown",
            "user_impact_level": "unknown",
            "clauses": [],
            "version": "unknown"
        }
    }

# -------------------------------
# Step 3: LLM-enhance chunks
# -------------------------------
def enrich_chunks(policy_text: str, source_url: str, company_name: str, domain: str):
    scrape_date = datetime.utcnow().isoformat() + "Z"
    base_metadata = {
        "text": f"{company_name} Policy Section",
        "source_url": source_url,
        "domain": domain,
        "scrape_date": scrape_date
    }

    raw_sections = split_policy_sections(policy_text)
    enriched = []

    for i, raw_chunk in enumerate(raw_sections):
        section_label = raw_chunk.strip().split("\n")[0] if "\n" in raw_chunk else f"Section {i}"
        document_input = build_input_chunk(raw_chunk, section_label, base_metadata)

        try:
            print(f"⏳ Enhancing chunk {i+1}/{len(raw_sections)}: {section_label[:40]}...")
            response = base_chain.invoke({"document": json.dumps(document_input)})
            parsed = json.loads(response)
            enriched.extend(parsed)
        except Exception as e:
            print(f"❌ Failed chunk {i}: {e}")
            continue

    return enriched

# -------------------------------
# Step 4: Run and save
# -------------------------------
if __name__ == "__main__":
    # === 1. Load your GitHub ToS text here ===
    policy_text = results[0]["content"]
    # === 2. Customize source metadata ===
    source_url = "https://docs.github.com/en/site-policy/github-terms/github-terms-of-service"
    company_name = "GitHub"
    domain = "github.com"

    # === 3. Enrich and Save ===
    enriched_chunks = enrich_chunks(policy_text, source_url, company_name, domain)

    print(f"✅ Done. {len(enriched_chunks)} enhanced chunks saved.")


/var/folders/m8/x0f9131s0bnfbnqp15hmpbpw0000gn/T/ipykernel_2526/4282388093.py:90: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  scrape_date = datetime.utcnow().isoformat() + "Z"


⏳ Enhancing chunk 1/55: GitHub Terms of Service - GitHub Docs...
❌ Failed chunk 0: Expecting value: line 1 column 1 (char 0)
⏳ Enhancing chunk 2/55: Section 1...
❌ Failed chunk 1: Expecting value: line 1 column 1 (char 0)
⏳ Enhancing chunk 3/55: Section 2...
❌ Failed chunk 2: Expecting value: line 1 column 1 (char 0)
⏳ Enhancing chunk 4/55: Section 3...
❌ Failed chunk 3: Expecting value: line 1 column 1 (char 0)
⏳ Enhancing chunk 5/55: Section 4...
❌ Failed chunk 4: Expecting value: line 1 column 1 (char 0)
⏳ Enhancing chunk 6/55: Section 5...
❌ Failed chunk 5: Expecting value: line 1 column 1 (char 0)
⏳ Enhancing chunk 7/55: Section 6...


KeyboardInterrupt: 